# Experiments 
To avoid long running times here we have modified the parameters (size of network, number of samples...). Next to them we give, in a comment, the value used for the paper's figures. 

In [8]:
import numpy as np
import pandas as pd
import networkx as nx
import os
from itertools import product

from tqdm import tqdm

In [9]:
# Moving to parent directory to import from CDFD and other_helpers
current_dir = os.getcwd()
os.chdir("..")
from CDFD import get_circularity, CDFD
from other_helpers import  uniform_multigraph, in_cycle_ratio, trophic_incoherence

# Moving back to original directory
os.chdir(current_dir)

## Figure 3

In [12]:
# Define parameters
n_max = 500 # 8000 (value used in paper)
n_division = 10 # 20 (value used in paper)
num_it = 10 # 50 (value used in paper)

n_values = [int(i) for i in np.logspace(np.log10(10), np.log10(n_max), n_division )] # uniformly spaced in log-plot
avr_weights = [1,2,3,4,5]
parameters = list(product (n_values, avr_weights))

In [13]:
# Generate random networks and compute their circularity
experiments = []

for _ in tqdm(range(num_it)):
    for n, avr_weight in parameters:
        experiment = {'num_nodes' : n, 'average_weight' : avr_weight}
        G = uniform_multigraph(n,n*avr_weight)
        C, D = CDFD(G , solution_method = "min_cost_pulp")
        experiment['max'] = get_circularity(C, D)
        C, D = CDFD(G , solution_method = "BFF")
        experiment['BFF'] = get_circularity(C, D)
        experiments.append(experiment)

exps = pd.DataFrame.from_dict(experiments)

100%|██████████| 10/10 [06:55<00:00, 41.54s/it]


In [15]:
# Saving data 
path_data = "data_figures"
if not os.path.exists(path_data):
    os.makedirs(path_data)
    
path_exps = os.path.join(path_data, "Figure3.csv")
exps.to_csv(path_exps, index=False) 

## Figure 4

In [18]:
# Define parameters
n = 100 # 2000 (value used in paper)
avr_weight_max = n
num_divisions = 20
num_it = 10 # 20 (value used in paper)

avr_weights = np.logspace(np.log10(1), np.log10(avr_weight_max), num_divisions ) # uniformly spaced in log-plot

In [19]:
# Generate random networks and compute their circularity
experiments = []

for _ in tqdm(range(num_it)):
    for avr_weight in avr_weights:
        experiment = {'num_nodes' : n, 'average_weight' : avr_weight}
        G = uniform_multigraph(n,int(n*avr_weight))
        C, D = CDFD(G , solution_method = "min_cost_pulp")
        experiment['max'] = get_circularity(C, D)
        C, D = CDFD(G , solution_method = "BFF")
        experiment['BFF'] = get_circularity(C, D)
        experiments.append(experiment)
exps = pd.DataFrame.from_dict(experiments)

100%|██████████| 10/10 [04:08<00:00, 24.85s/it]


In [20]:
# Saving data   
path_exps = os.path.join(path_data, "Figure4.csv")
exps.to_csv(path_exps, index=False) 

## Figure 5

### Random generated data

In [23]:
n_values = [20, 100, 500, 900] # [200, 1000, 5000, 9000] (values used in paper) 
exponent = -0.5
avr_degrees = [i**(1/exponent) for i in np.linspace(1**exponent,4**exponent,25)]
# From Figure 4 we expect this to create roughly uniform separation of BFF circularity

In [24]:
experiments = []
for n in tqdm(n_values): 
    for avr_degree in avr_degrees:
        experiment = {'num_nodes' : n, 'average_degree' : avr_degree, 'type' : 'Random'}
        G = uniform_multigraph(n, int(avr_degree*n))
        C, D = CDFD(G , solution_method = "min_cost_pulp")
        experiment ['max'] = get_circularity(C, D)
        C, D = CDFD(G , solution_method = "BFF")
        experiment ['BFF'] = get_circularity(C, D)
        experiment['incoherence'] = trophic_incoherence(G)
        experiment['All cycle edges'] = in_cycle_ratio(G)
        experiments.append(experiment)
exps_rnd = pd.DataFrame.from_dict(experiments)

100%|██████████| 4/4 [02:37<00:00, 39.41s/it]


### Sarafu data
The computation in this section may take a few minutes to finish. 

If one does not have access to this data you can still plot the rest of the data by removing exps_mon and exps_mod in the last cell of this notebook. 

FULL NETWORK

In [34]:
# Load data
sarafu_path = "data_sarafu"
flow_reg_dir = os.path.join(sarafu_path, "networks", "sarafu_reg_users.net")
flow_reg_nx = nx.DiGraph(nx.read_pajek(flow_reg_dir, encoding='UTF-8'))
print("Data loaded.")

# Compute circularity 
C, D = CDFD(flow_reg_nx , solution_method = "min_cost_pulp")
print( "Maximal circulairity of the full sarafu network is:", get_circularity(C, D))

C, D = CDFD(flow_reg_nx , solution_method = "BFF")
print( "BFF circulairity of the full sarafu network is:", get_circularity(C, D))

Data loaded.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\MathsCDT\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\MathsCDT\AppData\Local\Temp\ipykernel_11916\1223186361.py", line 8, in <module>
    C, D = CDFD(flow_reg_nx , solution_method = "min_cost_pulp")
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MathsCDT\Dropbox\matematiques_dropbox\Warwick\project directed networks\Code\Python\github\CDFD.py", line 137, in CDFD
    C, D = sol_method_dic[solution_method](W, TOL_ZERO = TOL_ZERO, TOL_decimals = TOL_decimals, MAX_decimals = MAX_decimals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MathsCDT\Dropbox\matematiques_dropbox\Warwick\project directed networks\Code\Python\github\CDFD.py", line 242, in CDFD_min_cost_pulp
    D = _min_cost_flow_pulp( W )
   

MONTHLY NETWORKS

In [ ]:
# Get monthly data
months = ['2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09',
          '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03','2021-04', '2021-05']

G_months = {}
for month in months: 
    flow_mod_dir = os.path.join(sarafu_path,"networks","monthly", "sarafu_reg_"+ month + ".net")
    graph = nx.DiGraph(nx.read_pajek(flow_mod_dir,encoding='UTF-8'))
    G_months[month] = graph

In [ ]:
# Compute monthly circularity
experiments = []
for month, G in G_months.items():
    experiment = {'num_nodes' : G.number_of_nodes(), 'average_weight' : np.sum(nx.adjacency_matrix(G).data)/G.number_of_nodes(),
                  'type' : 'Monthly data', 'average_degree' : G.number_of_edges()/G.number_of_nodes()}
    C, D = CDFD(G , solution_method = "min_cost_pulp")
    experiment ['max'] = get_circularity(C, D)
    C, D = CDFD(G , solution_method = "BFF")
    experiment ['BFF'] = get_circularity(C, D)
    experiment['incoherence'] = trophic_incoherence(G)
    experiment['All cycle edges'] = in_cycle_ratio(G)
    experiments.append(experiment)
exps_mon = pd.DataFrame.from_dict(experiments)

In [ ]:
# Get module data
path = os.path.join(sarafu_path,"networks","modules","mod_1")

G_modules = [] 
for file_name in os.listdir(path):
    G = nx.DiGraph(nx.read_pajek(os.path.join(path, file_name),encoding='UTF-8'))
    G_modules.append(G)

In [ ]:
# Compute modules circularity
experiments = []
for G in G_modules:
    experiment = {'num_nodes' : G.number_of_nodes(), 'average_weight' : np.sum(nx.adjacency_matrix(G).data)/G.number_of_nodes(),
                  'type' : 'Modules data', 'average_degree' : G.number_of_edges()/G.number_of_nodes()}
    C, D = CDFD(G , solution_method = "min_cost_pulp")
    experiment ['max'] = get_circularity(C, D)
    C, D = CDFD(G , solution_method = "BFF")
    experiment ['BFF'] = get_circularity(C, D)
    experiment['incoherence'] = trophic_incoherence(G)
    experiment['All cycle edges'] = in_cycle_ratio(G)
    experiments.append(experiment)
exps_mod = pd.DataFrame.from_dict(experiments)

SAVE DATA

In [ ]:
exps = pd.concat([exps_rnd, exps_mon, exps_mod], ignore_index=True)

path_exps = os.path.join(path_data, "Figure5.csv")
exps.to_csv(path_exps, index=False) 